# Pre-processing

In [1]:
import sys, os, glob, shutil
sys.path.append('../')

from MRIsegm.utils import get_slice, get_slice_info, get_rois, explore_slice, explore_roi, make_mask, mask_slice
from MRIsegm.processing import denoise_slice

from PIL import Image
from tqdm.notebook import trange



## Loading Data

In [2]:
src = '/Users/giuseppefilitto/Pazienti_anonym_sorted'

patients = os.listdir(src)
if '.DS_Store' in patients:
    patients.remove('.DS_Store')

bad_patients = [ 'BO9', 'BO17', 'BO28', 'BO36', 'BO37', 'BO39', 'BO40', 'BO54', 'BO72', 'BO77', 'BO86']

good_patients = list(set(patients) - set(bad_patients))

# ! Removing for test
good_patients.remove('BO38')
print("Number of good patients:",len(good_patients))

Number of good patients: 36


## Saving Train and Test images

### Training

In [3]:
progress_bar = trange(len(good_patients), desc="Progress")
for patient in good_patients:

    #! slices
    folder = 'T2'
    slice_path = os.path.join(src, patient, folder)

    if not os.path.isdir(slice_path):
        slice_path =  slice_path + "AX"

        if not os.path.isdir(slice_path):
            slice_path = os.path.join(src, patient, 'T25mm')

            if not os.path.isdir(slice_path):
                slice_path = os.path.join(src, patient, 't2DEF')


    slice_raw = get_slice(dir_path=slice_path)  

    alpha = 5
    slice = denoise_slice(slice_raw, alpha = alpha)

    roi_folder = 'T2ROI'
    roi_path = os.path.join(src, patient, roi_folder)

    #! ROIS
    roi = get_rois(roi_path=roi_path)
    

    slice_of_masks = mask_slice(slice=slice, rois=roi)
    

    positions = [roi[j].get('position') - 1 for j in range(len(roi))]

    print(f'patient: {patient}', end=', ', flush=True )
    for i in set(positions):

        img = Image.fromarray(slice[i, : ,:])
        filename = patient + '_layer_' + str(i) + '.png'
        dst = '../data/training/img/img'
        output =  os.path.join(dst, filename)

        if os.path.isfile(output):
            os.remove(output)
        
        img.save(output)

        mask = Image.fromarray(slice_of_masks[i, : ,:])
        filename = patient + '_mask_' + str(i) + '.png'
        dst = '../data/training/mask/img'
        output =  os.path.join(dst, filename)
        if os.path.isfile(output):
            os.remove(output)

        mask.save(output)
 
    progress_bar.update(1)    

progress_bar.close()    

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

patient: BO51, patient: BO76, patient: BO85, patient: BO71, patient: BO78, patient: BO75, patient: BO1, patient: BO33, patient: BO47, patient: BO90, patient: BO74, patient: BO31, patient: BO49, patient: BO45, patient: BO18, patient: BO11, patient: BO42, patient: BO32, patient: BO44, patient: BO50, patient: BO82, patient: BO26, patient: BO56, patient: BO29, patient: BO63, patient: BO66, patient: BO43, patient: BO35, patient: BO16, patient: BO52, patient: BO2, patient: BO60, patient: BO48, patient: BO68, patient: BO64, patient: BO61, 

### Validation

In [11]:
train_dir = '../data/training/img/img'
train_files = glob.glob(train_dir + '/*.png')

mask_dir = '../data/training/mask/img'
mask_files = glob.glob(mask_dir + '/*.png')



# ! validation
validation_dir = '../data/validation/img/img'
mask_validation_dir = '../data/validation/mask/img'

n_validation = 45
print("Training images:", len(train_files) - n_validation)
print("validation images:", n_validation)


Training images: 424
validation images: 45


In [12]:
for i in range(n_validation):
    
    dir, file = os.path.split(train_files[i])
    validation_img = os.path.join(validation_dir, file)

    img = train_files[i]
    if os.path.isfile(validation_img):
        os.remove(validation_img)

    shutil.move(img, validation_img)

    mask_dir = '../data/training/mask/img'
    mask_file = file.replace('_layer_','_mask_')
    
    mask = os.path.join(mask_dir, mask_file)
    mask_validation = os.path.join(mask_validation_dir, mask_file)
    if os.path.isfile(mask_validation):
        os.remove(mask_validation)

    shutil.move(mask, mask_validation)

### Test

In [15]:
test_patients = ['BO38']

for patient in test_patients:

   
    folders = ['T2AXAlta', 'T2AXBassa']
    roi_folders = ['T2ROIalta', 'T2ROIbassa']

    for k in range(len(folders)):


        slice_path = os.path.join(src, patient, folders[k])

        slice_raw = get_slice(dir_path=slice_path)  

        alpha = 5
        slice = denoise_slice(slice_raw, alpha = alpha)

        
        roi_path = os.path.join(src, patient, roi_folders[k])

        #! ROIS
        roi = get_rois(roi_path=roi_path)
        
        slice_of_masks = mask_slice(slice=slice, rois=roi)
        

        positions = [roi[j].get('position') - 1 for j in range(len(roi))]

        print(f'patient: {patient} folder: {folders[k]}', end=', ', flush=True )
        for i in set(positions):

            img = Image.fromarray(slice[i, : , :])
            filename = patient + '_' + str(folders[k]) + '_layer_' + str(i) + '.png'
            dst = '../data/test/img/img'
            output =  os.path.join(dst, filename)

            if os.path.isfile(output):
                os.remove(output)
            
            img.save(output)

            mask = Image.fromarray(slice_of_masks[i, : , :])
            filename = patient + '_' + str(roi_folders[k]) + '_mask_' + str(i) + '.png'
            dst = '../data/test/mask/img'
            output =  os.path.join(dst, filename)
            if os.path.isfile(output):
                os.remove(output)

            mask.save(output)



patient: BO38 folder: T2AXAlta, patient: BO38 folder: T2AXBassa, 